# Fetching Data

This notebook includes code that fetches data from balldontlie API and then saves it as a csv file.

In [1]:
import requests
import pandas as pd
import time

In [2]:
pd.set_option('display.max_columns', None)

NOTE: This make_request() function is different from the one in functions.py. It doesn't automatically detect if the request it a multi page request. (balldontlie API serves a maximum of 100 rows of data per request)

In [3]:
def make_request(endpoint, params=None, record_path=None, verbose=False):
    root = "https://www.balldontlie.io/api/v1/"
    response = requests.get(root + endpoint, params=params)
    if response.status_code != 200:
        print(response.status_code)
        return response
    if verbose: print("Success!")
    res = response.json()
    res = pd.json_normalize(res, record_path=record_path)
    return res

### game data

This data is only used for EDA purposes

In [40]:
game_data = make_request("games", params={"page":1, "per_page":100, "seasons":[2017,2018,2019,2020]}, record_path="data", verbose=True)

Success!


In [41]:
%%time
for i in range(2,501):
    if i%10 == 0: print(i)
    time.sleep(1.2)
    new_data = make_request("games", params={"page":i, "per_page":100, "seasons":[2017,2018,2019,2020]}, record_path="data")
    game_data = game_data.append(new_data)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
CPU times: user 41.9 s, sys: 2 s, total: 43.9 s
Wall time: 17min 51s


In [45]:
game_data.set_index("id", inplace=True)

In [51]:
game_data.to_csv("data/games.csv")

### stats data

This is the data used to build the model. Individual player stats for every NBA game since 1979.

The first block of code is just to get the meta data. To see how many pages of data we are going to have to request (as balldontlie API serves a maximum of 100 rows of data per request.)

In [123]:
# there are over 11000 pages!
all_stats_data_meta = make_request("stats", params={"page":1, "per_page":100}, record_path=None)

all_stats_data_meta

This code gets all 11483 pages of data from the API and saves it in a dataframe

In [6]:
stats_data = pd.DataFrame()

for i in range(1, 11483):
    # Print what page we're on every 10 pages to keep track of progress
    if i % 10 == 0:
        print(i)
        
    # Make sure not to exceed 60 API requests per minute (balldontlie API is free but limits request per minite)
    time.sleep(1.1)
    
    # Make the request and append to the dataframe
    new_data = make_request("stats", params={"page":i, "per_page":100}, record_path="data")
    stats_data = stats_data.append(new_data)
print("Done!")

10
Done!


This code saves the data to a csv file. It's commented out to not accidentally overwrite the current csv file as it took hours to pull all the data.

In [120]:
stats_data.set_index("id", inplace=True)

# stats_data.to_csv("data/stats_raw.csv")

In [126]:
for i in range(2, 11483):
    if i%100 == 0: print(i)
    time.sleep(1.5)
    new_data = make_request("stats", params={"page":i, "per_page":100}, record_path="data")
    all_stats_data = all_stats_data.append(new_data)
print("Done!")